# Определение языка и VK API

В данном задании вам нужно будет:

* используя API Вконтакте, скачать комментарии к первым ста постам из пяти сообществ
* натренировать модель распознавания языков на статьях из Википедии.
* распознать язык всех комментариев, где в тексте есть 10 и более символов, и построить статистику

## VK API

Для подключения к ВКонтакте мы будем использовать VK API. Здесь есть документация к этой библиотеке https://vk-api.readthedocs.io/en/latest/

In [3]:
!pip install vk_api

  Stored in directory: /Users/egor/Library/Caches/pip/wheels/f6/e9/d3/e1b33ba060d26168f9dbae48f1b5e83ed72b06a4931201ba00
Successfully built vk-api


In [4]:
import vk_api

# здесь нужно ввести данные своего аккаунта
# когда будете сдавать, не забудьте убрать эти две строчки из ноутбука
user = '' # вставьте сюда свой номер телефона
password = '' # вставьте сюда свой пароль

# авторизация
vk_session = vk_api.VkApi(login=user, password=password)
vk_session.auth()

vk = vk_session.get_api() # объект с API

Получить записи со страницы можно с помощью метода `wall.get`. Он принимает параметр `domain` — короткое имя пользователя или сообщества — и `count` — количество записей, которое вы хотите получить (максимум — 100). По Список методов для работы со стенами: https://vk.com/dev/wall

Например, вот так можно получить последние две записи с вот этой страницы https://vk.com/futureisnow. Выдача представляет собой словарь, в котором в поле `items` записан список словарей, содержащий информацию о каждой из записи. 

In [5]:
vk.wall.get(domain="futureisnow", count=2)

{'count': 8371,
 'items': [{'id': 256847,
   'from_id': -111587102,
   'owner_id': -111587102,
   'date': 1571045220,
   'marked_as_ads': 0,
   'post_type': 'post',
   'text': '',
   'signer_id': 183640804,
   'attachments': [{'type': 'photo',
     'photo': {'id': 457289659,
      'album_id': -7,
      'owner_id': -111587102,
      'user_id': 100,
      'sizes': [{'type': 'm',
        'url': 'https://sun1-16.userapi.com/c543107/v543107964/5c944/Xgs90Y4lvPI.jpg',
        'width': 97,
        'height': 130},
       {'type': 'o',
        'url': 'https://sun1-23.userapi.com/c543107/v543107964/5c949/iQp3zoxXW9w.jpg',
        'width': 130,
        'height': 173},
       {'type': 'p',
        'url': 'https://sun1-24.userapi.com/c543107/v543107964/5c94a/ZL3yDM2kM4Q.jpg',
        'width': 200,
        'height': 267},
       {'type': 'q',
        'url': 'https://sun1-17.userapi.com/c543107/v543107964/5c94b/F6ogOuZ6WfQ.jpg',
        'width': 320,
        'height': 427},
       {'type': 'r',
     

С помощью метода `groups.getById` можно получить информацию о сообществе, в том числе его id.

In [6]:
vk.groups.getById(group_ids="futureisnow")

[{'id': 111587102,
  'name': 'киберпанк, который мы заслужили',
  'screen_name': 'futureisnow',
  'is_closed': 0,
  'type': 'page',
  'is_admin': 0,
  'is_member': 0,
  'is_advertiser': 0,
  'photo_50': 'https://sun1-29.userapi.com/c845124/v845124938/4b879/-EzOFg-vjTg.jpg?ava=1',
  'photo_100': 'https://sun1-14.userapi.com/c845124/v845124938/4b878/Lr9NW4jLxOU.jpg?ava=1',
  'photo_200': 'https://sun1-85.userapi.com/c845124/v845124938/4b877/dqfgJ42Ir5E.jpg?ava=1'}]

Скачайте информацию о последних ста записях в следующих пабликах: https://vk.com/futureisnow, https://vk.com/eternalclassic, https://vk.com/ukrlit_memes, https://vk.com/ukrainer_net, https://vk.com/amanzohel, https://vk.com/barg_kurumk_culture

In [7]:
# список domain'ов, чтобы вам не копировать их самими :)
publics = ["futureisnow",
           "eternalclassic",
           "ukrlit_memes",
           "ukrainer_net",
           "amanzohel",
           "barg_kurumk_culture"]

In [21]:
items = {} # ключи — это паблики
for domain in publics:
    dirty_posts = vk.wall.get(domain=domain, count=100)
    items[domain] = dirty_posts['items']

{'futureisnow': [{'id': 256847, 'from_id': -111587102, 'owner_id': -111587102, 'date': 1571045220, 'marked_as_ads': 0, 'post_type': 'post', 'text': '', 'signer_id': 183640804, 'attachments': [{'type': 'photo', 'photo': {'id': 457289659, 'album_id': -7, 'owner_id': -111587102, 'user_id': 100, 'sizes': [{'type': 'm', 'url': 'https://sun1-16.userapi.com/c543107/v543107964/5c944/Xgs90Y4lvPI.jpg', 'width': 97, 'height': 130}, {'type': 'o', 'url': 'https://sun1-23.userapi.com/c543107/v543107964/5c949/iQp3zoxXW9w.jpg', 'width': 130, 'height': 173}, {'type': 'p', 'url': 'https://sun1-24.userapi.com/c543107/v543107964/5c94a/ZL3yDM2kM4Q.jpg', 'width': 200, 'height': 267}, {'type': 'q', 'url': 'https://sun1-17.userapi.com/c543107/v543107964/5c94b/F6ogOuZ6WfQ.jpg', 'width': 320, 'height': 427}, {'type': 'r', 'url': 'https://sun1-93.userapi.com/c543107/v543107964/5c94c/EsPE3zzE7fw.jpg', 'width': 510, 'height': 680}, {'type': 's', 'url': 'https://sun1-25.userapi.com/c543107/v543107964/5c943/E_dVxpxo

{'futureisnow': [{'id': 256847, 'from_id': -111587102, 'owner_id': -111587102, 'date': 1571045220, 'marked_as_ads': 0, 'post_type': 'post', 'text': '', 'signer_id': 183640804, 'attachments': [{'type': 'photo', 'photo': {'id': 457289659, 'album_id': -7, 'owner_id': -111587102, 'user_id': 100, 'sizes': [{'type': 'm', 'url': 'https://sun1-16.userapi.com/c543107/v543107964/5c944/Xgs90Y4lvPI.jpg', 'width': 97, 'height': 130}, {'type': 'o', 'url': 'https://sun1-23.userapi.com/c543107/v543107964/5c949/iQp3zoxXW9w.jpg', 'width': 130, 'height': 173}, {'type': 'p', 'url': 'https://sun1-24.userapi.com/c543107/v543107964/5c94a/ZL3yDM2kM4Q.jpg', 'width': 200, 'height': 267}, {'type': 'q', 'url': 'https://sun1-17.userapi.com/c543107/v543107964/5c94b/F6ogOuZ6WfQ.jpg', 'width': 320, 'height': 427}, {'type': 'r', 'url': 'https://sun1-93.userapi.com/c543107/v543107964/5c94c/EsPE3zzE7fw.jpg', 'width': 510, 'height': 680}, {'type': 's', 'url': 'https://sun1-25.userapi.com/c543107/v543107964/5c943/E_dVxpxo

{'futureisnow': [{'id': 256847, 'from_id': -111587102, 'owner_id': -111587102, 'date': 1571045220, 'marked_as_ads': 0, 'post_type': 'post', 'text': '', 'signer_id': 183640804, 'attachments': [{'type': 'photo', 'photo': {'id': 457289659, 'album_id': -7, 'owner_id': -111587102, 'user_id': 100, 'sizes': [{'type': 'm', 'url': 'https://sun1-16.userapi.com/c543107/v543107964/5c944/Xgs90Y4lvPI.jpg', 'width': 97, 'height': 130}, {'type': 'o', 'url': 'https://sun1-23.userapi.com/c543107/v543107964/5c949/iQp3zoxXW9w.jpg', 'width': 130, 'height': 173}, {'type': 'p', 'url': 'https://sun1-24.userapi.com/c543107/v543107964/5c94a/ZL3yDM2kM4Q.jpg', 'width': 200, 'height': 267}, {'type': 'q', 'url': 'https://sun1-17.userapi.com/c543107/v543107964/5c94b/F6ogOuZ6WfQ.jpg', 'width': 320, 'height': 427}, {'type': 'r', 'url': 'https://sun1-93.userapi.com/c543107/v543107964/5c94c/EsPE3zzE7fw.jpg', 'width': 510, 'height': 680}, {'type': 's', 'url': 'https://sun1-25.userapi.com/c543107/v543107964/5c943/E_dVxpxo

{'futureisnow': [{'id': 256847, 'from_id': -111587102, 'owner_id': -111587102, 'date': 1571045220, 'marked_as_ads': 0, 'post_type': 'post', 'text': '', 'signer_id': 183640804, 'attachments': [{'type': 'photo', 'photo': {'id': 457289659, 'album_id': -7, 'owner_id': -111587102, 'user_id': 100, 'sizes': [{'type': 'm', 'url': 'https://sun1-16.userapi.com/c543107/v543107964/5c944/Xgs90Y4lvPI.jpg', 'width': 97, 'height': 130}, {'type': 'o', 'url': 'https://sun1-23.userapi.com/c543107/v543107964/5c949/iQp3zoxXW9w.jpg', 'width': 130, 'height': 173}, {'type': 'p', 'url': 'https://sun1-24.userapi.com/c543107/v543107964/5c94a/ZL3yDM2kM4Q.jpg', 'width': 200, 'height': 267}, {'type': 'q', 'url': 'https://sun1-17.userapi.com/c543107/v543107964/5c94b/F6ogOuZ6WfQ.jpg', 'width': 320, 'height': 427}, {'type': 'r', 'url': 'https://sun1-93.userapi.com/c543107/v543107964/5c94c/EsPE3zzE7fw.jpg', 'width': 510, 'height': 680}, {'type': 's', 'url': 'https://sun1-25.userapi.com/c543107/v543107964/5c943/E_dVxpxo

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Найдите в документации (https://vk.com/dev/wall) метод для получения комментариев и получите первые сто комментариев каждого поста из выборки для каждого паблика.

In [42]:
dirty_corpora = {}
for domain in publics:
    dirty_corpora[domain] = []
    for clean_post in items[domain]:
        dirty_comms = vk.wall.getComments(owner_id=clean_post['owner_id'], post_id=clean_post['id'], count=100)
        dirty_corpora[domain].append(dirty_comms['items'])
        print('extraction of ' + str(len(dirty_corpora[domain])) + ' comment from ' + domain + ' finished')

extraction of 1 comment from futureisnow finished
extraction of 2 comment from futureisnow finished
extraction of 3 comment from futureisnow finished
extraction of 4 comment from futureisnow finished
extraction of 5 comment from futureisnow finished
extraction of 6 comment from futureisnow finished
extraction of 7 comment from futureisnow finished
extraction of 8 comment from futureisnow finished
extraction of 9 comment from futureisnow finished
extraction of 10 comment from futureisnow finished
extraction of 11 comment from futureisnow finished
extraction of 12 comment from futureisnow finished
extraction of 13 comment from futureisnow finished
extraction of 14 comment from futureisnow finished
extraction of 15 comment from futureisnow finished
extraction of 16 comment from futureisnow finished
extraction of 17 comment from futureisnow finished
extraction of 18 comment from futureisnow finished
extraction of 19 comment from futureisnow finished
extraction of 20 comment from futureisno

extraction of 59 comment from eternalclassic finished
extraction of 60 comment from eternalclassic finished
extraction of 61 comment from eternalclassic finished
extraction of 62 comment from eternalclassic finished
extraction of 63 comment from eternalclassic finished
extraction of 64 comment from eternalclassic finished
extraction of 65 comment from eternalclassic finished
extraction of 66 comment from eternalclassic finished
extraction of 67 comment from eternalclassic finished
extraction of 68 comment from eternalclassic finished
extraction of 69 comment from eternalclassic finished
extraction of 70 comment from eternalclassic finished
extraction of 71 comment from eternalclassic finished
extraction of 72 comment from eternalclassic finished
extraction of 73 comment from eternalclassic finished
extraction of 74 comment from eternalclassic finished
extraction of 75 comment from eternalclassic finished
extraction of 76 comment from eternalclassic finished
extraction of 77 comment fro

extraction of 16 comment from ukrainer_net finished
extraction of 17 comment from ukrainer_net finished
extraction of 18 comment from ukrainer_net finished
extraction of 19 comment from ukrainer_net finished
extraction of 20 comment from ukrainer_net finished
extraction of 21 comment from ukrainer_net finished
extraction of 22 comment from ukrainer_net finished
extraction of 23 comment from ukrainer_net finished
extraction of 24 comment from ukrainer_net finished
extraction of 25 comment from ukrainer_net finished
extraction of 26 comment from ukrainer_net finished
extraction of 27 comment from ukrainer_net finished
extraction of 28 comment from ukrainer_net finished
extraction of 29 comment from ukrainer_net finished
extraction of 30 comment from ukrainer_net finished
extraction of 31 comment from ukrainer_net finished
extraction of 32 comment from ukrainer_net finished
extraction of 33 comment from ukrainer_net finished
extraction of 34 comment from ukrainer_net finished
extraction o

extraction of 79 comment from amanzohel finished
extraction of 80 comment from amanzohel finished
extraction of 81 comment from amanzohel finished
extraction of 82 comment from amanzohel finished
extraction of 83 comment from amanzohel finished
extraction of 84 comment from amanzohel finished
extraction of 85 comment from amanzohel finished
extraction of 86 comment from amanzohel finished
extraction of 87 comment from amanzohel finished
extraction of 88 comment from amanzohel finished
extraction of 89 comment from amanzohel finished
extraction of 90 comment from amanzohel finished
extraction of 91 comment from amanzohel finished
extraction of 92 comment from amanzohel finished
extraction of 93 comment from amanzohel finished
extraction of 94 comment from amanzohel finished
extraction of 95 comment from amanzohel finished
extraction of 96 comment from amanzohel finished
extraction of 97 comment from amanzohel finished
extraction of 98 comment from amanzohel finished
extraction of 99 com

In [45]:
clean_corpora = {}
for domain in publics:
    clean_corpora[domain] = []
    for comments in corpora[domain]:
        for comment in comments:
            try:
                if len(comment['text']) > 10:
                    clean_corpora[domain].append(comment['text'])
                else:
                    continue
            except KeyError:
                continue
print(clean_corpora)

{'futureisnow': ['фу у вас на фотке писька', 'День первый \nХакер приходит в общественную столовую и с возмущением обнаруживает, что солонку на столе может открутить кто попало и насыпать туда что угодно. Хакер приходит домой и пишет гневное письмо директору столовой: "Я, meG@Duc, обнаружил уязвимость солонки в Вашей столовой. Злоумышленник может вскрыть солонку и насыпать туда яду! Примите меры срочно!" \n\nДень второй \nДиректор среди прочих деловых писем, запросов о поставках еды и курьерских уведомлений получает письмо, и пожимает плечами: "Кому этот бред только в голову пришёл?" \n\nДень пятый \nХакер приходит в столовую, насыпает во все солонки яду. Погибает триста человек, директора три месяца таскают по судам и, в конце концов, оправдывают за отсутствием состава преступления. Хакер пишет письмо в стиле "ну что, видали?". \n\nДень 96-ой \nДиректор покупает специально спроектированные солонки с кодовым замком. Посетители столовой чувствуют, что они в этой жизни чего-то не понимаю

**Задание со звездочкой:** вы могли заметить, что если обращаться к каждому посту отдельно, то все занимает довольно продолжительное время (около пяти минут). Найдите в документации vk_api способ сделать это быстрее.

In [27]:
# я могу сделать это с помощью хранимых процедур ВК и execute, тогда за одно обращение к методу будет 25 
# запросов в API, но, мне кажется, здесь ожидается другой ответ.

## Тренировка моделей

В наших комментариях встречались русский, украинский, английский и бурятский.

In [37]:
langs = {'ru', 'uk', 'en', 'bxr'}
from collections import Counter
import nltk
import wikipedia

[nltk_data] Downloading package punkt to /Users/egor/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Скачайте документы, на которых вы будете обучать свои модели. Для наших целей хорошо иметь для каждого языка корпус размером около 50 статей.

In [32]:
def get_texts_for_lang(lang, n=10):
    wiki_content = []
    wikipedia.set_lang(lang)
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
            wiki_content.append("%s\n%s" % (page.title, page.content.replace('=', '')))
        except wikipedia.exceptions.WikipediaException:
            print("Skip %s" % page_name)
    return wiki_content

wiki_texts = {}
for lang in langs:
    wiki_texts[lang] = get_texts_for_lang(lang, 50)
    print(lang, len(wiki_texts[lang]))

//anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file //anaconda3/lib/python3.7/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Skip Татаринцев, Александр Владимирович
Skip Павлюк
Skip Софоново
Skip Приключения Травки (значения)
ru 46
Skip З'єднання
Skip Каракудуцький сільський округ
Skip Борисова
uk 47
Skip Wolf (TV series)
Skip Kekelaokalani
en 48
bxr 50


Сделайте определялку на частотах слов.

In [38]:
def collect_freqlist(wiki_pages, max_len=100):
    freqlist = Counter()
    for text in wiki_pages:
        for word in nltk.word_tokenize(text.lower()):
            if word.isalpha():
                freqlist[word] += 1
    return dict(freqlist.most_common(max_len))

freq_lists = {}
for lang in langs:
    freq_lists[lang] = collect_freqlist(wiki_texts[lang])
    
def simple_lang_detect(freq_lists, text):
    counts = Counter()
    for lang, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
            counts[lang] += int(freq_list[word] > 0)
    return counts.most_common()

Сделайте определялку на символьных энграммах.

In [39]:
from sklearn import feature_extraction
from sklearn import pipeline
from sklearn import naive_bayes
import numpy as np

clf = pipeline.Pipeline([
    ('vctr', feature_extraction.text.TfidfVectorizer(ngram_range=(1, 2), analyzer='char')),
    ('clf', naive_bayes.MultinomialNB())
])

all_texts = []
lang_indices = []
for lang in wiki_texts:
    all_texts.extend(wiki_texts[lang])
    lang_indices.extend([lang]*len(wiki_texts[lang]))
    
# Обучаем классификатор
clf.fit(np.array(all_texts), np.array(lang_indices))
clf.predict(all_texts)

array(['ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru',
       'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru',
       'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru',
       'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru', 'ru',
       'ru', 'ru', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk',
       'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk',
       'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk',
       'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'uk', 'en', 'uk', 'uk',
       'uk', 'uk', 'uk', 'uk', 'uk', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'bxr', 'bxr',
       'bxr', 'bxr', 'bxr', 'bxr', 'bxr', 'bxr', 

# Определение языка

Определите язык каждого комментария в каждом паблике с помощью определялки на частотах слов и покажите доли языков среди комментариев для каждого паблика.

In [55]:
percs = {}
for public in publics:
    lang_ratio = Counter()
    for comment in clean_corpora[public]:
        lang = simple_lang_detect(freq_lists, comment)
        try:
            if lang[0][1] == 0:
                continue
            if 'ru' in lang[0]:
                lang_ratio['ru'] += 1
            elif 'en' in lang[0]:
                lang_ratio['en'] += 1
            elif 'uk' in lang[0]:
                lang_ratio['uk'] += 1
            elif 'bxr' in lang[0]:
                lang_ratio['bxr'] += 1
        except IndexError:
            continue

    for language, entries in lang_ratio.items():
        percs[language] = str((entries / sum(lang_ratio.values(), 2)*100)) + '%'
    print(public, percs)

futureisnow {'ru': '95.51451187335093%', 'en': '1.58311345646438%', 'uk': '2.638522427440633%'}
eternalclassic {'ru': '7.4074074074074066%', 'en': '92.15686274509804%', 'uk': '2.638522427440633%'}
ukrlit_memes {'ru': '41.66666666666667%', 'en': '2.564102564102564%', 'uk': '54.48717948717948%'}
ukrainer_net {'ru': '34.375%', 'en': '2.564102564102564%', 'uk': '64.0625%'}
amanzohel {'ru': '61.53846153846154%', 'en': '2.564102564102564%', 'uk': '2.564102564102564%', 'bxr': '30.76923076923077%'}
barg_kurumk_culture {'ru': '81.10236220472441%', 'en': '2.564102564102564%', 'uk': '0.7874015748031495%', 'bxr': '16.535433070866144%'}


Сделайте то же самое для определителя на символьных энграммах.

In [57]:
percs = {}
for public in publics:
    lang_ratio = Counter(clf.predict(clean_corpora[public]))

    for language, entries in lang_ratio.items():
        percs[language] = str((entries / sum(lang_ratio.values(), 2)*100)) + '%'
    print(public, percs)

futureisnow {'uk': '1.4178482068390326%', 'ru': '93.16096747289407%', 'en': '3.669724770642202%', 'bxr': '1.58465387823186%'}
eternalclassic {'uk': '1.4178482068390326%', 'ru': '93.16096747289407%', 'en': '99.73009446693656%', 'bxr': '1.58465387823186%'}
ukrlit_memes {'uk': '55.37848605577689%', 'ru': '38.645418326693225%', 'en': '2.3904382470119523%', 'bxr': '2.788844621513944%'}
ukrainer_net {'uk': '70.1492537313433%', 'ru': '22.388059701492537%', 'en': '2.4875621890547266%', 'bxr': '3.9800995024875623%'}
amanzohel {'uk': '70.1492537313433%', 'ru': '14.785992217898833%', 'en': '2.4875621890547266%', 'bxr': '84.43579766536965%'}
barg_kurumk_culture {'uk': '0.8403361344537815%', 'ru': '57.56302521008403%', 'en': '0.42016806722689076%', 'bxr': '40.33613445378151%'}


Обсудите работу каждого из классификаторов, обсудите ошибки, объясните разницу в результатах.

Определение языка по символьным энграммам имеет погрешность при работе на языках с похожим алфавитом, что можно заметить 
на статистике: при определении по частотным словам бурятских комментариев не было найдено в первых двух пабликах, и их там скорее
всего нет, но есть русский. В целом, очевидно, что перввый классификатор лучше работает с большим объемом текста, второй же
менее зависим от размера выборки. Для представленной задачи, мне кажется, лучше подходит энграммный метод определения языка.